In [ ]:
# find the first link to extract

if number_ep_database == 0:
    link  = url
else:
    # Extract last link in database
    try:
            # Execute a query
        cursor.execute(f"SELECT link from {table_name} order by Episod desc limit 1")
        link = cursor.fetchall()
        link = str(link[0][0])
    except mysql.connector.Error as err:
        print(f"Error: {err}")
           
    soup = create_soup(link)
    link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']
    link = extract_link(link)

In [ ]:
# create lists
season_list = []
episod_season_list = []
episod_list= []
title_list = []
rating_list = []
day_list= []
month_list= []
year_list = []
link_list = []

link_list.append(link)


for i in range(batch_ep_to_extract):

    try:

        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link}")
        soup = create_soup(link)
        
        # Season and Episod
        ep = soup.find('div', class_="sc-3f4e3993-0 fYpskP").text
        season,episod = extract_episod(ep)
        episod_list.append(episod)
        
        # Title
        title = extract_title(soup.find('span', class_="hero__primary-text").text)
        
        title_list.append(title)

        #Rating
        rate = soup.find('span', class_="sc-c4ffe080-1 iQZtLP").text.replace(",",".")
        rating_list.append(float(rate))

        # Date
        date = soup.find_all('li', class_="ipc-inline-list__item")
        day,month,year = extract_date(date)
        day_list.append(day)
        month_list.append(month)
        year_list.append(year)

        #Link 
        if i == 0 and number_ep_database == 0: 
            link = soup.find('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase').attrs['href']
        else:
            link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']

        if i < batch_ep_to_extract - 1:        
            link = extract_link(link)
            link_list.append(link)

        i = i+1

    except IndexError:
        pass         

    except WebDriverException as E: 
        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link} failed")



In [ ]:
# As the season are wrong in IMDB website, Extract season from wikipedia page

# Import Requests
import requests
from bs4 import BeautifulSoup

wiki_url = "https://en.wikipedia.org/wiki/Lists_of_One_Piece_episodes"
listofseason = []
nb_ep_season = []

# Download the contents of the webpage in text format
data  = requests.get(wiki_url).text 

# Create a Beautiful soup of the data
soup = BeautifulSoup(data, "html.parser")


# Extract the tableau
table = soup.find_all("table", {"class": "wikitable plainrowheaders"})[0]

# Extract the list of seasons
links = table.find_all('a')
for link in links:
    listofseason.append(int(link.text)) 

# Extract the list of episods per season
trs = table.find_all('tr')
for i in trs[2:]:
    tds = i.find_all('td')
    nb_ep_season.append(int(tds[1].text))

for i in range(1,(len(listofseason)+1)):
    for j in range(1,(nb_ep_season[i-1]+1)):
        episod_season_list.append(j)
        season_list.append(i)

episod_season_list =  episod_season_list[number_ep_database:(number_ep_database+batch_ep_to_extract)]
season_list =  season_list[number_ep_database:(number_ep_database+batch_ep_to_extract)]

In [17]:
if len(link_list) > len(episod_list):
    del link_list[-1]

In [ ]:
data = {"Episod":episod_list,"Season":season_list,"Episod_season":episod_season_list, "Title":title_list, "Day":day_list, "Month":month_list, "Year":year_list, "Rate":rating_list, "link":link_list}
df = pd.DataFrame(data)
df.to_csv(csv_file,index=False,header=False, mode= "a")

In [19]:
if df.shape[0] == batch_ep_to_extract:
    write_to_file(log_file,f"Extracting {batch_ep_to_extract} episod successfuly")
else:
    write_to_file(log_file,f"Extraction failed")

In [20]:
write_to_file(log_file,f"Sending {len(df)} rows to MySQL table '{table_name}'.")
try:
    # Create a list of column names from the DataFrame
    columns = ', '.join(df.columns)

    # Create a placeholders string for the SQL query
    placeholders = ', '.join(['%s'] * len(df.columns))

    # Create the SQL query
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the SQL query with the DataFrame values
    cursor.executemany(sql, df.values.tolist())

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file,f"Successfully sent {len(df)} rows to MySQL table '{table_name}'.")
except mysql.connector.Error as e:
    print(f"Error sending data to MySQL: {e}")